# **Import Libraries**

In [ ]:
import pandas as pd 

# **Read Data**

In [ ]:
# قراءة البيانات
df = pd.read_csv("Road Accident Data.csv")
df

# **Data Understanding**

In [ ]:
# نظرة عامة على البيانات
df.info()
df.describe()


In [ ]:
print(df.isnull().sum())


## **عرض تكرار بعض الأعمدة لفهم القيم**


In [ ]:
df['Junction_Control'].value_counts()

In [ ]:
df['Junction_Detail'].value_counts()

In [ ]:
df['Road_Surface_Conditions'].value_counts()

In [ ]:
df['Road_Type'].value_counts()

In [ ]:
df['Vehicle_Type'].value_counts()

In [ ]:
df['Light_Conditions'].value_counts()

In [ ]:
df['Carriageway_Hazards'].value_counts()

In [ ]:
df['Weather_Conditions'].value_counts()

In [ ]:
# تحويل التاريخ والوقت

df['Accident Date'] = pd.to_datetime(df['Accident Date'])

df['Time'] = pd.to_datetime(df['Time'], format='%H:%M', errors='coerce').dt.time


# **Data Cleaning**

In [ ]:
# إزالة التكرارات الكاملة
df = df.drop_duplicates()


## **Fill Nulls**

In [ ]:
# ترتيب البيانات حسب التاريخ

df = df.sort_values(by='Accident Date')
df


In [ ]:
# تعويض القيم المفقودة في الأعمدة
# الطقس (تعبئة بالتتابع)

df['Weather_Conditions'] = df['Weather_Conditions'].fillna(method='ffill')
print("num of nulls :", df['Weather_Conditions'].isnull().sum())


In [ ]:
# الوقت (تعبئة بالقيمة الأكثر تكرارًا)

most_common_time = df['Time'].mode()[0]
df['Time'] = df['Time'].fillna(most_common_time)
print("num of nulls :", df['Time'].isnull().sum())

In [ ]:
# نوع الطريق (تعبئة بالقيمة الأكثر تكرارًا)

most_common_road = df['Road_Type'].mode()[0]
df['Road_Type'] = df['Road_Type'].fillna(most_common_road)

print("num of nulls :", df['Road_Type'].isnull().sum())


In [ ]:
# المخاطر (تعبئة ب"لا يوجد خطر")

df['Carriageway_Hazards'] = df['Carriageway_Hazards'].fillna('No hazard')
print("num of nulls :", df['Carriageway_Hazards'].isnull().sum())



In [ ]:

# استخراج الفصل من تاريخ الحادث
def get_season(month, day):
    if (month == 3 and day >= 21) or (4 <= month <= 6) or (month == 6 and day <= 20):
        return 'Spring'
    elif (month == 6 and day >= 21) or (7 <= month <= 9) or (month == 9 and day <= 20):
        return 'Summer'
    elif (month == 9 and day >= 21) or (10 <= month <= 12) or (month == 12 and day <= 20):
        return 'Autumn'
    else:
        return 'Winter'

# تطبيق دالة الفصول على البيانات
df['Season'] = df['Accident Date'].apply(lambda x: get_season(x.month, x.day))

# ملء القيم الفاضية بناءً على الفصل (من خلال الأكثر تكرارًا لكل فصل)
df['Road_Surface_Conditions'] = df.groupby('Season')['Road_Surface_Conditions'].transform(lambda x: x.fillna(x.mode()[0]))

# عرض الصفوف التي تحتوي على قيم مفقودة بعد التعبئة
print(df[df['Road_Surface_Conditions'].isnull()])
print("num of nulls :", df['Road_Surface_Conditions'].isnull().sum())



In [ ]:
# التأكد من عدم وجود قيم مفقودة
print(df.isnull().sum())

## **Handle Outliers**

In [ ]:
#التعامل مع outliers
Q1 = df['Number_of_Casualties'].quantile(0.25)
Q3 = df['Number_of_Casualties'].quantile(0.75)
IQR = Q3 - Q1

# تحديد الـ outliers
outliers = df[(df['Number_of_Casualties'] < (Q1 - 1.5 * IQR)) | (df['Number_of_Casualties'] > (Q3 + 1.5 * IQR))]

print("عدد القيم الشاذة:", len(outliers))
print(outliers[['Accident_Index', 'Number_of_Casualties']])


In [ ]:
# استبدال القيم الشاذة بالقيمة الوسيطة (Median)

median_value = df['Number_of_Casualties'].median()
df['Number_of_Casualties'] = df['Number_of_Casualties'].apply(lambda x: median_value if x in outliers['Number_of_Casualties'].values else x)


In [ ]:
Q1_vehicles = df['Number_of_Vehicles'].quantile(0.25)
Q3_vehicles = df['Number_of_Vehicles'].quantile(0.75)
IQR_vehicles = Q3_vehicles - Q1_vehicles

# استخراج القيم الشاذة بناءً على IQR لعمود Number_of_Vehicles
outliers_vehicles = df[(df['Number_of_Vehicles'] < (Q1_vehicles - 1.5 * IQR_vehicles)) | (df['Number_of_Vehicles'] > (Q3_vehicles + 1.5 * IQR_vehicles))]

# عرض القيم الشاذة في عمود Number_of_Vehicles
print("عدد القيم الشاذة في Number_of_Vehicles:", len(outliers_vehicles))
print(outliers_vehicles[['Accident_Index', 'Number_of_Vehicles']])

In [ ]:
# استبدال القيم الشاذة في Number_of_Vehicles بالقيمة الوسيطة (Median)
median_vehicles = df['Number_of_Vehicles'].median()
df['Number_of_Vehicles'] = df['Number_of_Vehicles'].apply(lambda x: median_vehicles if x in outliers_vehicles['Number_of_Vehicles'].values else x)



# **Feature Engineering**

**Create columns (Weekend - Peak Hours - Time Split)**

## **Weekend**

In [ ]:
# Create Weekend column
df['Is_Weekend'] = df['Accident Date'].dt.dayofweek >= 5  # 5 = Saturday, 6 = Sunday


## **Peak hours**

In [ ]:
# Create Peak hour column

def is_peak_hour(time):
    # تحويل الوقت إلى ساعة
    hour = time.hour
    # ساعات الذروة من 7:00 AM إلى 9:00 AM ومن 4:00 PM إلى 6:00 PM
    if (7 <= hour < 9) or (16 <= hour < 18):
        return True
    else:
        return False

# إضافة عمود 'is_peak_hour' بناءً على وقت الحادث
df['is_peak_hour'] = df['Time'].apply(is_peak_hour)

## **Time slot**

In [ ]:
# دالة لتصنيف الساعة إلى فترات زمنية في اليوم:

def get_time_slot(hour):
    if 0 <= hour < 6:            # - Late Night: من منتصف الليل حتى 6 صباحًا
        return 'Late Night'
    elif 6 <= hour < 12:         # - Morning: من 6 صباحًا حتى 12 ظهرًا
        return 'Morning'
    elif 12 <= hour < 18:        # - Afternoon: من 12 ظهرًا حتى 6 مساءً
        return 'Afternoon'
    else:                        # - Evening: من 6 مساءً حتى منتصف الليل
        return 'Evening'
    
# - Evening: من 6 مساءً حتى منتصف الليل
df['Time Slot'] = df['Time'].dt.hour.apply(get_time_slot)

# **More columns for easier visualization**

## **تبسيط نوع المركبة**

قمت بتبسيط القيم في عمود "Vehicle_Type" لتسهيل التحليل عن طريق تصنيف المركبات إلى فئات مثل "Car"، "Motorcycle"، و"Bus".


In [ ]:
vehicle_map = {
    'Car': 'Car',
    'Taxi/Private hire car': 'Car',
    'Van / Goods 3.5 tonnes mgw or under': 'Light Van',
    'Goods over 3.5t. and under 7.5t': 'Medium Van',
    'Goods 7.5 tonnes mgw and over': 'Heavy Van',
    'Bus or coach (17 or more pass seats)': 'Bus',
    'Minibus (8 - 16 passenger seats)': 'Minibus',
    'Pedal cycle': 'Bicycle',
    'Motorcycle 50cc and under': 'Motorcycle',
    'Motorcycle 125cc and under': 'Motorcycle',
    'Motorcycle over 125cc and up to 500cc': 'Motorcycle',
    'Motorcycle over 500cc': 'Motorcycle',
    'Agricultural vehicle': 'Agricultural vehicle',
    'Ridden horse': 'Horse',
    'Other vehicle': 'Other'
}

# إضافة عمود جديد فيه المسميات المبسطة
df['Vehicle Class'] = df['Vehicle_Type'].map(vehicle_map)



## **تبسيط حالة الطقس**

قمت بتبسيط القيم في عمود "Weather_Conditions" لتصنيف حالات الطقس إلى فئات أبسط مثل "Clear Weather"، "Rainy Weather"، و"Snowy Weather".


In [ ]:
# إنشاء دالة لتحويل القيم في عمود 'Weather_Conditions'
weather_map = {
    'Fine no high winds': 'Clear Weather',
    'Raining no high winds': 'Rainy Weather',
    'Raining + high winds': 'Rainy Weather',
    'Snowing no high winds': 'Snowy Weather',
    'Snowing + high winds': 'Snowy Weather',
    'Fine + high winds': 'Windy Weather',
    'Fog or mist': 'Foggy',
    'Other': 'Other'
    }

# إضافة عمود جديد مع القيم المبسطة
df['Weather'] = df['Weather_Conditions'].map(weather_map)


## **تبسيط القيم في عمود 'Junction_Control'**

تم استخدام قاموس لتبسيط القيم المختلفة في عمود 'Junction_Control' بحيث تسهل الفهم والتحليل، مع إضافة عمود جديد يعرض القيم المبسطة.


In [ ]:
# إنشاء قاموس لتبسيط القيم في عمود 'Junction_Control'
junction_control_map = {
    'Give way or uncontrolled': 'Uncontrolled',
    'Data missing or out of range': 'Unknown',
    'Auto traffic signal': 'Traffic Signal',
    'Not at junction or within 20 metres': 'Not at Junction',
    'Stop sign': 'Stop Sign',
    'Authorised person': 'Authorized Person'
}

# إضافة عمود جديد مع القيم المبسطة
df['Junction_Control_Simplified'] = df['Junction_Control'].map(junction_control_map)

# عرض القيم الجديدة بعد التبسيط
print(df['Junction_Control_Simplified'].unique())


## **تبسيط القيم في عمود 'Junction_Detail'**

تم استخدام قاموس لتبسيط القيم المختلفة في عمود 'Junction_Detail' بحيث تسهل الفهم والتحليل، مع إضافة عمود جديد يعرض القيم المبسطة.


In [ ]:
# إنشاء قاموس لتبسيط القيم في عمود 'Junction_Detail'
junction_detail_map = {
    'Not at junction or within 20 metres': 'No Junction',
    'T or staggered junction': 'T-Junction',
    'Crossroads': 'Crossroads',
    'Roundabout': 'Roundabout',
    'Private drive or entrance': 'Private Drive',
    'Other junction': 'Other Junction',
    'Slip road': 'Slip Road',
    'More than 4 arms (not roundabout)': 'Multi-directional Junction',
    'Mini-roundabout': 'Mini-Roundabout'
}

# إضافة عمود جديد مع القيم المبسطة
df['Junction_Type'] = df['Junction_Detail'].map(junction_detail_map)

# عرض القيم الجديدة بعد التبسيط
print(df['Junction_Type'].unique())


## **تبسيط القيم في عمود 'Light_Conditions'**

تم استخدام قاموس لتبسيط القيم في عمود 'Light_Conditions' لتحسين وضوح البيانات وتسهيل التحليل، مع إضافة عمود جديد يعرض القيم المبسطة.


In [ ]:
# إنشاء قاموس لتبسيط القيم في عمود 'Light_Conditions'
light_conditions_map = {
    'Daylight': 'Daylight',
    'Darkness - lights lit': 'Lights Lit',
    'Darkness - no lighting': 'No Lighting',
    'Darkness - lighting unknown': 'Lighting Unknown',
    'Darkness - lights unlit': 'Lights Unlit'
}

# إضافة عمود جديد مع القيم المبسطة
df['Light Status'] = df['Light_Conditions'].map(light_conditions_map)

# عرض القيم الجديدة بعد التبسيط
print(df['Light Status'].unique())


# **convert columns to categorical data**

In [ ]:
# تحويل بعض الأعمدة إلى فئات (Categorical)
cat_cols = ['Day_of_Week', 'Month', 'Accident_Severity', 'Weather', 'Vehicle Class',
            'Road_Type', 'Junction_Control_Simplified', 'Junction_Type', 'Light Status',
            'Season', 'Time Slot', 'is_peak_hour', 'Is_Weekend']
for col in cat_cols:
    df[col] = df[col].astype('category')


# **EXPORT DATA**

In [52]:
df.to_csv('Cleaned_Road_Accident_Data.csv', index=False)
